# Applied Data Science Capstone Project (part 1)

## 1. Introduction 
### 1.1. Description of the problem and discussion of the background
The purpose of this project is to determine the best district in San Francisco for a person that may want to live in 
such city. The evaluation metrics for the problem considered are:
1. Safety, which is related to the total number of crimes corresponding to each district;
2. Attractiveness, which is assessed by listing the ten most common venues in each neighborhood.

The above features have been chosen because they have a relevant impact in determining the choice of a new location for
a given individual.

Lastly, each neighborhood will be clustered using a procedure based on **DBSCAN**, such that neighborhoods with
similar characteristics (from a safety and attractiveness point of view) are assigned the same label. 

### 1.2. Description of the data and how it will be used to solve the problem
The crime dataset used in this work corresponds to all the crimes that occurred in San Francisco in 2016. It can be freely
downloaded from Kaggle website at the following link: https://www.kaggle.com/roshansharma/sanfranciso-crime-dataset

The crime dataset, displayed in Section 1.2.1, is structured in a table. The column labels have the following meaning:
1. **IncidntNum**: incident number;
2. **Category**: category of crime;
3. **Descript**: description of the crime;
4. **DayOfWeek**: day of the week in which the crime occurred;
5. **Date**: date in which the crime occurred;
6. **Time**: time in which the crime occurred;
7. **PdDistrict**: Police department district;
8. **Resolution**: kind of punishment given to the criminal to resolve the case;
9. **Address**: address where the crime scene happened;
10. **X**: longitude of the crime location;
11. **Y**: latitude of the crime location;
12. **Location**: Exact location (latitude, longitude);
13. **PdId**: Pd ID.

Lastly, the venues related to each district are retrieved from Foursquare, using the same procedure displayed in the
assignments of the previous weeks
#### 1.2.1. Preprocessing crime data
##### Importing libraries and loading the data into a dataframe

In [1]:
import pandas as pd
import os
import numpy as np
import requests
from pytictoc import TicToc
from geopy.geocoders import Nominatim

In [2]:
absolute_path = os.path.abspath(os.path.dirname('Data/'))
df = pd.read_csv(absolute_path + "\Crime_SF.csv")
print(f"Shape of the raw dataframe: {df.shape}")
df.head()

Shape of the raw dataframe: (150500, 13)


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


##### Selection of the columns of interest

In [3]:
df2 = pd.DataFrame(df[['PdDistrict', 'Category', 'X', 'Y']])
df2.sort_values(by=['PdDistrict', 'Category'], inplace=True)
df2.reset_index(drop=True, inplace=True)
# The last row needs to be removed, since that crime has not been assigned to any district (NaN)
df2.tail()

,PdDistrict,Category,X,Y
150495,TENDERLOIN,WEAPON LAWS,-122.411966,37.784914
150496,TENDERLOIN,WEAPON LAWS,-122.412054,37.781614
150497,TENDERLOIN,WEAPON LAWS,-122.416711,37.783357
150498,TENDERLOIN,WEAPON LAWS,-122.416711,37.783357
150499,NaN,LARCENY/THEFT,-122.413352,37.708202


##### Removing the last row, which contains a NaN value

In [4]:
df2.drop(df2.tail(1).index, inplace=True)
df2.tail()


,PdDistrict,Category,X,Y
150494,TENDERLOIN,WEAPON LAWS,-122.409661,37.786439
150495,TENDERLOIN,WEAPON LAWS,-122.411966,37.784914
150496,TENDERLOIN,WEAPON LAWS,-122.412054,37.781614
150497,TENDERLOIN,WEAPON LAWS,-122.416711,37.783357
150498,TENDERLOIN,WEAPON LAWS,-122.416711,37.783357


##### Crime dataset information

In [5]:
print(f"Current shape of the dataframe: {df2.shape}")
print("-------------------------------------------")
print(df2.info())
# Total number of crimes in each Police district
print("-------------------------------------------")
print("District      Number of crimes\n")
print(df2['PdDistrict'].value_counts())
print("-------------------------------------------")
# Total number of crimes per category
print("Category\t\t       Number of occurrencies\n")
print(df2['Category'].value_counts())
print("-------------------------------------------")
print(f"Number of Police districts = {len(df2['PdDistrict'].unique())}.")
print(f"Number of different crimes = {len(df2['Category'].unique())}.")

Current shape of the dataframe: (150499, 4)
-------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 150499 entries, 0 to 150498
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   PdDistrict  150499 non-null  object 
 1   Category    150499 non-null  object 
 2   X           150499 non-null  float64
 3   Y           150499 non-null  float64
dtypes: float64(2), object(2)
memory usage: 5.7+ MB
None
-------------------------------------------
District      Number of crimes

SOUTHERN      28445
NORTHERN      20100
MISSION       19503
CENTRAL       17666
BAYVIEW       14303
INGLESIDE     11594
TARAVAL       11325
TENDERLOIN     9942
RICHMOND       8922
PARK           8699
Name: PdDistrict, dtype: int64
-------------------------------------------
Category		       Number of occurrencies

LARCENY/THEFT                  40408
OTHER OFFENSES                 19599
NON-CRIMINAL               

##### Pivoting the table to observe the number of crimes for each category in each district 

In [6]:
temp = pd.DataFrame(df2)
temp.insert(2, "", np.ones(df2.shape[0]))

In [7]:
df2_pivot = pd.pivot_table(temp, values=[""], index=['Category'], columns=['PdDistrict'], aggfunc=np.sum, fill_value=0)
df2_pivot['Total'] = df2_pivot.sum(axis=1)
df2_pivot.columns = df2_pivot.columns.map(''.join)
df2_pivot.head()

,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN,Total
Category,,,,,,,,,,,
ARSON,71,29,22,46,27,13,14,33,18,13,286
ASSAULT,1775,1187,1506,2110,1536,524,473,2352,918,1196,13577
BAD CHECKS,4,3,2,2,4,2,5,6,6,0,34
BRIBERY,20,3,8,10,4,1,2,8,4,6,66
BURGLARY,521,645,534,793,803,413,395,842,695,161,5802


#### 1.2.2. Data retrieval from Foursquare
##### Definition of Foursquare credentials

In [8]:
ID = '1B2QEZLDCQCQUAXR325SRGV0T1YT4FPSQLIJLYBBGTBEHIKE'
secret = 'CUVST2LHRHS1LHKBXH4SPMH1C3H3P5FBVIFLNFSSH4GZFDF2' 
version = '20200401'
limit = 500
print(f"Credentials\nCLIENT ID:     {ID}\nCLIENT SECRET: {secret}")
print('Developer credentials:')

Credentials
CLIENT ID:     1B2QEZLDCQCQUAXR325SRGV0T1YT4FPSQLIJLYBBGTBEHIKE
CLIENT SECRET: CUVST2LHRHS1LHKBXH4SPMH1C3H3P5FBVIFLNFSSH4GZFDF2
Developer credentials:


##### Extraction of Foursquare data

In [9]:
# This function retrieves the nearby venues for a given district in San Francisco
def getNearbyVenues(names, LAT, LON, radius=3500):
    venues_list=[]
    for i, j, k in zip(names, LAT, LON):
        #  API request URL
        url = f"https://api.foursquare.com/v2/venues/explore?&client_id={ID}&client_secret={secret}&v={version}" \
              f"&ll={j},{k}&radius={radius}&limit={limit}"
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # Return only relevant information for each nearby venue
        venues_list.append([(i, j, k, v['venue']['name'], v['venue']['location']['lat'], 
            v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue',
                             'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return nearby_venues


##### San Francisco Police dept. addresses (available on: https://sfgov.org/policecommission/police-district-maps), used as "districts' center points"

In [10]:
address_dict = {
    'Bayview': '201 Williams Avenue',
    'Central': '766 Vallejo Street',
    'Ingleside': 'Havelock St',
    'Mission': '630 Valencia Street',
    'Northern': '1125 Fillmore Street',
    'Park': '1899 Waller Street',
    'Richmond': '461 6th Avenue',
    'Southern': '1251 3rd Street',
    'Taraval': '2345 24th Avenue',
    'Tenderloin': '301 Eddy Street',
}
# Coordinates of the districts
geo_locator = Nominatim(user_agent="my_username")
lat = []
lon = []
for _ in address_dict.values():
    loc = geo_locator.geocode(_ + ", San Francisco, CA, 94122")
    lat.append(loc.latitude)
    lon.append(loc.longitude)

##### Extraction of San Francisco venues for each district in the database

In [11]:
t = TicToc()
t.tic()
venues = getNearbyVenues(names=df2['PdDistrict'].unique(), LAT=lat, LON=lon)
t.toc("All venues successfully extracted. Elapsed time:")

All venues successfully extracted. Elapsed time: 7.236550 seconds.


##### General information about the extracted dataframe and number of different venue categories

In [12]:
print(f"Shape of the venues dataframe: {venues.shape}")
print(f"There are {len(venues['Venue Category'].unique())} different types of venues.")
venues.groupby('Neighborhood').count().head()

Shape of the venues dataframe: (1000, 7)
There are 181 different types of venues.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BAYVIEW,100,100,100,100,100,100
CENTRAL,100,100,100,100,100,100
INGLESIDE,100,100,100,100,100,100
MISSION,100,100,100,100,100,100
NORTHERN,100,100,100,100,100,100


##### One-hot encoding of venues

In [13]:
venues_df = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
venues_df['PdDistrict'] = venues['Neighborhood']
pos = [venues_df.columns[-1]] + list(venues_df.columns[:-1])
venues_df = venues_df[pos]

In [14]:
# Rows are then grouped by neighborhood and the values corresponding to each category are obtained by computing the
# total number of occurrence of each type of venue
venues_df = venues_df.groupby('PdDistrict').sum().reset_index()
print(f"Original dataframe shape: {venues.shape}.")
print(f"Transformed dataframe shape: {venues_df.shape}.")
venues_df.head(10)



Original dataframe shape: (1000, 7).
Transformed dataframe shape: (10, 182).


,PdDistrict,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Trail,Turkish Restaurant,Udon Restaurant,Vietnamese Restaurant,Waterfall,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,BAYVIEW,0,0,1,1,0,0,0,1,0,...,0,1,0,1,0,0,0,2,0,0
1,CENTRAL,0,0,0,0,0,0,0,0,3,...,2,0,0,1,0,6,1,2,0,1
2,INGLESIDE,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,1,1,2,0,0
3,MISSION,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,4,0,8,0,0
4,NORTHERN,0,1,0,1,0,0,0,0,1,...,1,0,0,0,0,3,0,5,0,0
5,PARK,1,0,0,0,0,1,0,1,1,...,1,0,0,0,1,0,0,2,0,0
6,RICHMOND,1,0,0,0,1,1,0,0,1,...,1,0,0,1,1,1,0,2,0,0
7,SOUTHERN,0,0,0,0,0,0,0,1,2,...,0,0,0,2,0,0,3,2,0,0
8,TARAVAL,0,0,0,0,0,0,0,0,0,...,2,0,1,2,0,1,1,1,1,0
9,TENDERLOIN,0,0,0,1,0,0,0,0,3,...,0,0,0,1,0,6,2,2,0,0
